In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix


In [18]:
df = pd.read_csv('exams.csv')

In Naive Bayes, all varialbes must be discrete functions. Since the scores are continous variables, it is neccesary to bin the results into sets. This set will be based on a fail(0-54) or pass(55-100). An average score will be added to test the overall effectiveness of the preperation test. If a student average is over 55%, it will be considered a pass.

In [19]:
labels = ['fail','pass']
bins = [0,55,101]
df['average score'] = (df['math score'] + df['writing score']+df['reading score'])/3
df['math score'] = pd.cut(df['math score'],bins = bins, labels = labels, include_lowest = True)
df['reading score'] = pd.cut(df['reading score'],bins = bins, labels = labels, include_lowest = True)
df['writing score'] = pd.cut(df['writing score'],bins = bins, labels = labels, include_lowest = True)
df['average score'] = pd.cut(df['average score'],bins = bins, labels = labels, include_lowest = True)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score
0,male,group A,high school,standard,completed,pass,pass,pass,pass
1,female,group D,some high school,free/reduced,none,fail,pass,fail,fail
2,male,group E,some college,free/reduced,none,pass,pass,fail,pass
3,male,group B,high school,standard,none,pass,pass,pass,pass
4,male,group E,associate's degree,standard,completed,pass,pass,pass,pass


In [26]:
le = preprocessing.LabelEncoder()
for col in df.columns:
        df[col] = le.fit_transform(df[col])
        
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score
0,1,0,2,1,0,1,1,1,1
1,0,3,5,0,1,0,1,0,0
2,1,4,4,0,1,1,1,0,1
3,1,1,2,1,1,1,1,1,1
4,1,4,0,1,0,1,1,1,1


Values are now encoded and ready to be used in GaussianNB

## Cross validation

Split the data into the factors and target. Dataset is further split into a training and dataset with a 7:3 ratio

In [27]:
factors = df.iloc[:,0:5]
target = df.iloc[:,5:9]
X_train, X_test, y_train, y_test = train_test_split(
   factors, target, test_size = 0.3, random_state = 1
)


## Training

In [43]:
gnb = GaussianNB()
gnb.fit(X_train, y_train['average score'])
y_pred = gnb.predict(X_test)
print("Accuracy score is : ",metrics.accuracy_score(y_test['average score'], y_pred_average))

Accuracy score is :  0.8066666666666666


In [44]:
y_pred
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [45]:
x = 'hellosdadafgasgsdhfsdhsdfhfsd'

In [36]:
y = x + " " + "hello"

In [37]:
print(y)

hellosdadafgasgsdhfsdhsdfhfsd hello


In [ ]:
def insert_space(string, integer):
    return string[0:integer] + ' ' + string[integer:]

t = ""

i = 0
while i < len(x)/10: 
    t = t + insert_space(x[i*10:],10) 
print(t)

In [ ]:
y = insert_space(x,10)
r = y + insert_space(x[10:],10)
print(r)

In [30]:
x = insert_space(x,4)
print(x)

he l lo


In [50]:
print (insert_space(x[10:],10))

fgasgsdhfs dhsdfhfsd


In [1]:
import statistics
import numpy as np

In [61]:
N = 500
n = 12
y = [897,992,752,1093,768,828,1428,1152,875,642,909,1122]
x = [815,919,690,984,500,560,1323,1067,789,573,834,1049]
ys = 954.8
xs = 841.9
sy= 216
sx= 242.4
b = ys/xs
xu = 880
yr = b*xu
bo = 238.8
b1 = .8505


In [62]:
yreg = bo+b1*xu
print(yreg)

987.24


In [63]:
k = 0
for i in range(12):
    ei = (y[i]-bo-b1*x[i])
    k = k + np.square(ei)
    
se = k/11
v = (1-(12/500))*(se/12)


In [67]:
print(yr - 1.96*np.sqrt(v))

962.1225463201919


In [65]:
print(se)

4121.798721977271


In [66]:
print(v)

335.2396293874847
